In [1]:
import pydantic as pd2
import pydantic as pd
import pydantic.v1 as pd1
import sys

In order to understand the performance cost of using a docs schema, let's write some tests:

In [2]:
class MyBaseClass(pd.BaseModel):
    a: str = ""
    """
    Example Description
    """
    
    b: str = pd.Field(default="", description="Example Description")

our_class = MyBaseClass()
our_class

MyBaseClass(a='', b='')

In [3]:
# Calculate the memory cost
a_class_attribute_memory_cost = sys.getsizeof(our_class.a)
b_class_attribute_memory_cost = sys.getsizeof(our_class.b)
print(f"a_class_attribute memory: {a_class_attribute_memory_cost}")
print(f"b_class_attribute memory: {b_class_attribute_memory_cost}")

a_class_attribute memory: 49
b_class_attribute memory: 49


So there's a few options of what we can do here. We could create a higher-dimensional `pd.Field` that can be compiled into a better documentation representation which is more complete.

In [4]:
example_default = ""
example_basic_field = pd.Field(default="")

Let's analyse the memory aspect of this data representation:

In [5]:
# Calculate the memory cost
default_string_memory_cost = sys.getsizeof(example_default)
default_field_memory_cost = sys.getsizeof(example_basic_field)
print(f"a_class_attribute memory: {default_string_memory_cost}")
print(f"b_class_attribute memory: {default_field_memory_cost}")

a_class_attribute memory: 49
b_class_attribute memory: 216


So we can clearly tell there's a difference here. It's interesting because we can tell that even though the actual declaration on its own does have 4x higher memory cost, at the instantiation default level it is equivalent.

So if anything we can look inside annotations of each class and compile the documentation from there.

In [6]:
example_basic_field.__annotations__

{'annotation': 'type[Any] | None',
 'default': 'Any',
 'default_factory': 'typing.Callable[[], Any] | None',
 'alias': 'str | None',
 'alias_priority': 'int | None',
 'validation_alias': 'str | AliasPath | AliasChoices | None',
 'serialization_alias': 'str | None',
 'title': 'str | None',
 'field_title_generator': 'typing.Callable[[str, FieldInfo], str] | None',
 'description': 'str | None',
 'examples': 'list[Any] | None',
 'exclude': 'bool | None',
 'discriminator': 'str | types.Discriminator | None',
 'deprecated': 'Deprecated | str | bool | None',
 'json_schema_extra': 'JsonDict | typing.Callable[[JsonDict], None] | None',
 'frozen': 'bool | None',
 'validate_default': 'bool | None',
 'repr': 'bool',
 'init': 'bool | None',
 'init_var': 'bool | None',
 'kw_only': 'bool | None',
 'metadata': 'list[Any]',
 'metadata_lookup': 'ClassVar[dict[str, typing.Callable[[Any], Any] | None]]'}

The problem with this is that it only contains the types of the annotations, it does not contain the actual meaning. We can verify if this is the same for the attribute declaration

In [7]:
# our_class.a.__annotations__ # Doesn't work
# our_class.b.__annotations__ # Doesn't work
our_class.__annotations__

{'a': str, 'b': str}

So it is concievable we can overwrite the annotations of a class attribute. However, it is interesting that the field description is not directly accessible from the class instantiation.

So we can improve the documentation based on assigning this type of elements. We could extract the class definition from there in principle. However, clearly we have an issue which is where does the actual documentation data get contained.

We can extract the json schema defaults and variables from the `model_json_schema` attribute of the pydantic class.

In [9]:
from pydantic.fields import FieldInfo
from pydantic import BaseModel

def check_json_schema_extra(field: FieldInfo) -> bool:
    """Check if the FieldInfo contains a 'json_schema_extra' parameter."""
    return hasattr(field, 'json_schema_extra') and field.json_schema_extra is not None



def determine_pydantic_version_from_base_model(model: BaseModel):
    """Determine if a BaseModel is from Pydantic v1 or v2."""
    if hasattr(model, 'model_fields'):
        return 2
    elif hasattr(model, '__fields__'):
        return 1
    else:
        raise ValueError("Unknown Pydantic version or incompatible BaseModel class.")


def get_field_infos(model: BaseModel):
    """Get all FieldInfo instances from a Pydantic model, compatible with v1 and v2."""
    version = determine_pydantic_version_from_base_model(model)

    field_infos = []

    # Handle Pydantic v2
    if version == 2:
        for field_name, field in model.model_fields.items():
            field_infos.append(field)

    # Handle Pydantic v1
    elif version == 1:
        for field_name, field in model.__fields__.items():
            field_infos.append(field)

    return field_infos

our_class.model_json_schema()

{'properties': {'a': {'default': '', 'title': 'A', 'type': 'string'},
  'b': {'default': '',
   'description': 'Example Description',
   'title': 'B',
   'type': 'string'}},
 'title': 'MyBaseClass',
 'type': 'object'}

So when we go and define the parameters we want on our parameter table:

```
Units
Description
Type
Equation
```

We need to then evaluate how to define this in a standardized way, and where this gets extracted from.

So we've been analyzing the `__annotations__` and the provided json schema, it can be seen that a `pydantic.Field` contains hte example description and relevant information in the way we want. So it makes sense to create a higher dimensional version of the `pydantic.Field` and use this to annotate. We can then extract the information from there and compile each field in the particular format enabled by the extension. So we provide a full cycle of the implementation strategy.

In [ ]:
# help(pd.fields.Field)
pd.fields.Field(extra={
    "hello": 1
})

In [ ]:
test_extra_fieldinfo = pd.fields.FieldInfo(test=1)
dir(test_extra_fieldinfo)

In [ ]:
help(pd.fields.FieldInfo)

In [18]:
class PhysicalParameterV1(pd1.BaseModel):
    units: str = ""

class PhysicalParameterV2(pd2.BaseModel):
    units: str = ""

In [15]:
# help(pd.fields.Field)
pd.fields.Field(extra=PhysicalParameter())

NameError: name 'PhysicalParameter' is not defined

In [16]:
get_field_infos(PhysicalParameterV1)

[ModelField(name='units', type=str, required=False, default='')]

In [17]:
get_field_infos(PhysicalParameterV2)

[FieldInfo(annotation=str, required=False, default='')]

In [23]:
class PhysicalFieldInfo(pd.fields.FieldInfo):
    a: PhysicalParameterV1

In [12]:
PhysicalFieldInfo()

PhysicalFieldInfo(annotation=NoneType, required=True)

In [25]:
PhysicalFieldInfo()

PhysicalFieldInfo(annotation=NoneType, required=True)